In [4]:
!pip install opencv-python
!pip install mediapipe
!pip install streamlit


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import mediapipe as mp 
import time
import cv2 
import csv
import os
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import joblib

# Testing Mediapipe

In [3]:
mp_drawing = mp.solutions.drawing_utils 
mp_holistic = mp.solutions.holistic

In [10]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # BGR - RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False       
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Process
        results = holistic.process(image)
        
        image_height, image_width, _ = image.shape
        
        # RGB - BGR
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
       
        
        # Hand Detection
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        print(results.right_hand_landmarks)
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

None
None
None
None
None
None
None
None
landmark {
  x: 0.11088132858276367
  y: 1.0720765590667725
  z: 9.282790074394143e-07
}
landmark {
  x: 0.20776423811912537
  y: 1.0236855745315552
  z: -0.06497933715581894
}
landmark {
  x: 0.2695924639701843
  y: 0.9368147850036621
  z: -0.09819583594799042
}
landmark {
  x: 0.30436480045318604
  y: 0.8479242920875549
  z: -0.12394752353429794
}
landmark {
  x: 0.35375338792800903
  y: 0.7710139751434326
  z: -0.1520707756280899
}
landmark {
  x: 0.1879175901412964
  y: 0.808650016784668
  z: -0.06171625480055809
}
landmark {
  x: 0.2045794278383255
  y: 0.688508927822113
  z: -0.09182658046483994
}
landmark {
  x: 0.210215762257576
  y: 0.6084185242652893
  z: -0.11868871748447418
}
landmark {
  x: 0.21461153030395508
  y: 0.54246985912323
  z: -0.14016525447368622
}
landmark {
  x: 0.1288415789604187
  y: 0.7997698783874512
  z: -0.051624566316604614
}
landmark {
  x: 0.12161967158317566
  y: 0.6662001609802246
  z: -0.07843107730150223
}
l

# Capture Picture for Dataset

In [4]:
landmarks = ['class']
for val in range(1, 42 + 1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val)]
    
with open('dataset.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [7]:
class_name = "A"
cap = cv2.VideoCapture(0)
data = 0
with mp_holistic.Holistic(min_detection_confidence=0.3,min_tracking_confidence=0.3) as holistic:
    while cap.isOpened():
        while (data<=250):
            ret, frame = cap.read()
            # BGR 2 RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Flip on horizontal
            image = cv2.flip(image, 1)
            # Set flag
            image.flags.writeable = False
            # Detections
            results = holistic.process(image)
            # Set flag to true
            image.flags.writeable = True
            # RGB 2 BGR
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            
            lh = list(np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3))
            rh = list(np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3))

            row = lh+rh
            # Adda class name 
            row.insert(0, class_name)

            # Export to CSV
            with open('dataset.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 

            cv2.imshow('Ambil Gambar', image)

            if cv2.waitKey(10) & 0xFF == ord('x'):
                break
                
            data=data+1
        break
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


# Data Training from CSV

In [2]:
df = pd.read_csv('dataset.csv')

In [3]:
col = df.groupby('class')['class'].count()
print (col)
df.shape

Series([], Name: class, dtype: int64)


(0, 127)

In [9]:
df.head(5)

,class,x1,y1,z1,x2,y2,z2,x3,y3,z3,...,z39,x40,y40,z40,x41,y41,z41,x42,y42,z42


In [7]:
X = df.drop('class', axis=1) # features akan menjadi semua nilai koord
y = df['class'] # target = label kelas

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

ValueError: With n_samples=0, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [30]:
#Mengimport Model yang pembuat gunakan yaitu RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix,roc_curve, auc, accuracy_score


#Memanggil Fungsi Model
RFC = RandomForestClassifier()
#fitting Model terhadap data training
RFC.fit(X_train, y_train)
y_pred = RFC.predict(X_test)
#Akurasi dari Data training
print(accuracy_score(y_test,y_pred)) #mencetak akurasi prediksi
print(confusion_matrix(y_test,y_pred)) #mencetak confusion Matrix
print(classification_report(y_test,y_pred)) #mencetak classification report Precision, Recall, F1

0.9205298013245033
[[66 12]
 [ 0 73]]
              precision    recall  f1-score   support

           A       1.00      0.85      0.92        78
           B       0.86      1.00      0.92        73

    accuracy                           0.92       151
   macro avg       0.93      0.92      0.92       151
weighted avg       0.93      0.92      0.92       151



In [33]:
joblib.dump(RFC, "RFC_model.sav")

['RFC_model_Ardava.sav']

# Testing

In [14]:
RFC = joblib.load("RFC_model.sav")

In [23]:
cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        curr_time = time.time()
        
        ret, frame = cap.read()

        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Flip on horizontal
        image = cv2.flip(image, 1)

        # Set flag
        image.flags.writeable = False

        # Detections
        results = holistic.process(image)
        
        # get image shape
        image_height, image_width, _ = image.shape

        # Set flag to true
        image.flags.writeable = True

        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        if results.right_hand_landmarks or results.left_hand_landmarks:
            # Mengambil Pose landmarks
            lh = list(np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3))
            rh = list(np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3))

            # Satukan baris
            row = lh+rh

            # Tambah class name 
            X = pd.DataFrame([row])
            hand_class = RFC.predict(X)[0]
            
            cv2.rectangle(image, (0,0), (120, 40), (245, 117, 16), -1)
            cv2.putText(image, 'CLASS',(10,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, hand_class.split(' ')[0], (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('x'):
            break

cap.release()
cv2.destroyAllWindows()

c:\Users\fatan\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
c:\Users\fatan\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
c:\Users\fatan\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
c:\Users\fatan\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
c:\Users\fatan\AppDa